##### Basic analysis of fit correlation parameters from the correlation task.

Updated for eLife v2. Now run on vrh (full) model, deprecating analyses that don't make sense for that model.

Produces analyses associated with Fig 7.

Normative evidence weighting and accumulation in correlated environments. Tardiff et al. (2025).

In [1]:
#clear memory
rm(list=ls())

## loading data/libraries ##

#load libraries
library(lme4)
library(dplyr)
library(lmerTest)
library(blme)

se <- function(x) sqrt(var(x) / length(x))

switch(Sys.info()[['sysname']],
       Windows = PROJECT_DIR <- paste0('C:/Users/',Sys.getenv('USERNAME'),
                              '/Dropbox/Goldlab/correlated/'),
       Darwin = PROJECT_DIR <- '~/Dropbox/Goldlab/correlated/'
)

DATA_DIR = paste0(PROJECT_DIR,'/data/')
setwd(PROJECT_DIR)


DATA_FILE = 'rho_params_best_2024-11-20.csv'

Loading required package: Matrix


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [2]:
# functions we need
fisherz <- function(r){
    .5*log((1+r)/(1-r))
}

fishezr <- function(r){
    (exp(2*z)-1)/(exp(2*z)+1)
}

In [3]:
rho_df <- read.table(paste0(DATA_DIR,DATA_FILE),sep=',', header=TRUE, 
                    stringsAsFactors=FALSE,na.strings = 'NaN')

if (any(is.na(rho_df))) {
    stop('Misisng data detected!')
}

head(rho_df)

,rho_cond,subject,param,value,rho,scale_dev
,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,0.2,5bd781291fd7c80001bb1fad,Rn,-0.1762526,-0.2,1.0147336
2,0.2,5bd781291fd7c80001bb1fad,Rp,0.1605802,0.2,0.9834379
3,0.2,5d645bf6912c630018e269e3,Rn,-0.1073971,-0.2,1.0562924
4,0.2,5d645bf6912c630018e269e3,Rp,0.2635020,0.2,1.0261181
5,0.2,5e705a1be6c65a62c56a3143,Rn,-0.1073450,-0.2,1.0563232
6,0.2,5e705a1be6c65a62c56a3143,Rp,0.2080705,0.2,1.0033571


In [4]:
# convert correlations to fisher-z scale
rho_df$rhoz <- fisherz(rho_df$rho)
rho_df$valuez <- fisherz(rho_df$value)

rho_df$devz <- rho_df$valuez-rho_df$rhoz

## What is the slope of objective z(rho) ~ subjective z(rho)

In [5]:
#test slope
rho.lm.0 <- lmer(valuez~rhoz + (1|subject), 
                 data=rho_df, 
                 control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

In [6]:
#subjects significantly underestimate correlation
summary(rho.lm.0)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: valuez ~ rhoz + (1 | subject)
   Data: rho_df
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

REML criterion at convergence: -146.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.5556 -0.4029 -0.0162  0.4632  4.7508 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.006512 0.08069 
 Residual             0.020801 0.14423 
Number of obs: 200, groups:  subject, 100

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)    
(Intercept) -0.04869    0.01300 99.00000  -3.744 0.000304 ***
rhoz         0.70759    0.01477 99.00000  47.919  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
     (Intr)
rhoz 0.000 

## test deviation of objective rho from subjective rho
Is subjective z(rho) - objective z(rho) significantly different from zero?

Since positive and negative correlation differences move in opposite directions, we flip the sign of the negative correlation differences.

In [7]:
rho_df$devz_ab <- rho_df$devz
rho_df$devz_ab[rho_df$rho<0] <- -rho_df$devz_ab[rho_df$rho<0]

In [8]:
rho_df$rho.fs <- as.factor(rho_df$rho)
contrasts(rho_df$rho.fs) <- contr.sum(8)
contrasts(rho_df$rho.fs)

rho_df$param.fs <- as.factor(rho_df$param)
contrasts(rho_df$param.fs) <- contr.sum(2)
contrasts(rho_df$param.fs)

my_simple<-contr.treatment(2)-matrix(rep(1/2, 2), ncol=1)
rho_df$param.f <- as.factor(rho_df$param)
contrasts(rho_df$param.f) <- my_simple
contrasts(rho_df$param.f)
#rho_df[c('rho','rho.fs')]

-0.8,1,0,0,0,0,0,0
-0.6,0,1,0,0,0,0,0
-0.4,0,0,1,0,0,0,0
-0.2,0,0,0,1,0,0,0
0.2,0,0,0,0,1,0,0
0.4,0,0,0,0,0,1,0
0.6,0,0,0,0,0,0,1
0.8,-1,-1,-1,-1,-1,-1,-1


Rn,1
Rp,-1


,2
Rn,-0.5
Rp,0.5


In [9]:
#NOT separated by pos/neg corr
rhof.lm.0 <- lmer(devz_ab~1 + (1|subject), 
                 data=rho_df, 
                 control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

boundary (singular) fit: see help('isSingular')



In [10]:
summary(rhof.lm.0)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: devz_ab ~ 1 + (1 | subject)
   Data: rho_df
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

REML criterion at convergence: -84

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5212 -0.3777  0.1558  0.5628  4.2493 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.00000  0.0000  
 Residual             0.03739  0.1934  
Number of obs: 200, groups:  subject, 100

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)  -0.18165    0.01367 199.00000  -13.29   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
optimizer (bobyqa) convergence code: 0 (OK)
boundary (singular) fit: see help('isSingular')


### gut checks because of singularity
Because the intercept random effect variance was zero (boundary issue), we verify the results in two ways.

1) placeing a weakly informative prior on the random effects
2) repeating with multiple optimizers

See: https://bbolker.github.io/mixedmodels-misc/glmmFAQ.html#singular-models-random-effect-variances-estimated-as-zero-or-correlations-estimated-as---1 and the help page for lme4 isSingular().

In [11]:
#do we gain any insight into singularity by using a prior on covariance?
rhof.lm.b <- blmer(devz_ab~1 + (1|subject), 
                 data=rho_df, 
                 control=lmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))

In [12]:
summary(rhof.lm.b)

Cov prior  : subject ~ wishart(df = 3.5, scale = Inf, posterior.scale = cov, common.scale = TRUE)
Prior dev  : 3.6689

Linear mixed model fit by REML ['blmerMod']
Formula: devz_ab ~ 1 + (1 | subject)
   Data: rho_df
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

REML criterion at convergence: -83

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4090 -0.3927  0.1454  0.5459  4.1451 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.003007 0.05483 
 Residual             0.034702 0.18629 
Number of obs: 200, groups:  subject, 100

Fixed effects:
            Estimate Std. Error t value
(Intercept) -0.18165    0.01427  -12.73

In [21]:
rhof.lm.0.all <- allFit(rhof.lm.0)
summary(rhof.lm.0.all)

bobyqa : 

boundary (singular) fit: see help('isSingular')



[OK]
Nelder_Mead : 

boundary (singular) fit: see help('isSingular')



[OK]
nlminbwrap : 

...

boundary (singular) fit: see help('isSingular')



[OK]
nloptwrap.NLOPT_LN_BOBYQA : 

boundary (singular) fit: see help('isSingular')



[OK]


$which.OK
                       bobyqa                   Nelder_Mead 
                         TRUE                          TRUE 
                   nlminbwrap nloptwrap.NLOPT_LN_NELDERMEAD 
                         TRUE                          TRUE 
    nloptwrap.NLOPT_LN_BOBYQA 
                         TRUE 

$msgs
$msgs$bobyqa
[1] "boundary (singular) fit: see help('isSingular')"

$msgs$Nelder_Mead
[1] "boundary (singular) fit: see help('isSingular')"

$msgs$nlminbwrap
[1] "boundary (singular) fit: see help('isSingular')"

$msgs$nloptwrap.NLOPT_LN_NELDERMEAD
[1] "boundary (singular) fit: see help('isSingular')"

$msgs$nloptwrap.NLOPT_LN_BOBYQA
[1] "boundary (singular) fit: see help('isSingular')"


$fixef
                              (Intercept)
bobyqa                         -0.1816459
Nelder_Mead                    -0.1816459
nlminbwrap                     -0.1816459
nloptwrap.NLOPT_LN_NELDERMEAD  -0.1816459
nloptwrap.NLOPT_LN_BOBYQA      -0.1816459

$llik
                   

### finally compute sds for rho deviations

In [14]:
rho_sd_rho <- summarise(group_by(rho_df,rho,param),rho_sd=sd(valuez))
rho_sd_rho

summarise(group_by(rho_sd_rho,param),rho_sd_mean=mean(rho_sd))

`summarise()` has grouped output by 'rho'. You can override using the `.groups`
argument.


rho,param,rho_sd
<dbl>,<chr>,<dbl>
-0.8,Rn,0.09476144
-0.6,Rn,0.09694017
-0.4,Rn,0.09590433
-0.2,Rn,0.08688486
0.2,Rp,0.11281864
0.4,Rp,0.16323730
0.6,Rp,0.26723474
0.8,Rp,0.24714026


param,rho_sd_mean
<chr>,<dbl>
Rn,0.0936227
Rp,0.1976077
